In [15]:
# RentTheRunway
import json
import pandas as pd

def filter_by_freq(df: pd.DataFrame, column: str, min_freq: int) -> pd.DataFrame:
    freq = df[column].value_counts()
    frequent_values = freq[freq > min_freq].index
    return df[df[column].isin(frequent_values)]
def create_id_map(sampled_ids):
    sampled_ids = sorted(sampled_ids)
    id_map = {x:i for i,x in enumerate(sampled_ids)}
    return id_map
def print_sample_summary(df, user_col_name='userId', item_col_name='movieId'):
    n_unique_users = len(set(df[user_col_name]))
    n_unique_items = len(set(df[item_col_name]))
    n_ratings = len(df)
    print(n_ratings, " ratings, ", n_unique_users, " users, ", n_unique_items, " items")
    print("Sparsity: ", n_ratings/(n_unique_users*n_unique_items))

lines = []
for line in open('../datasets/renttherunway_final_data.json', 'r').readlines():
    lines.append(json.loads(line))

In [16]:
df = pd.DataFrame(lines)
df = df.drop(['fit', 'bust size', 'weight', 'rented for', 'review_text', 'body type', 
              'review_summary', 'category', 'height', 'size', 'age', 'review_date'], axis=1)
print_sample_summary(df, 'user_id', 'item_id')

192544  ratings,  105571  users,  5850  items
Sparsity:  0.0003117665293831097


In [21]:
filtered_df = filter_by_freq(df, 'user_id', 30)
print_sample_summary(filtered_df, 'user_id', 'item_id')


6035  ratings,  106  users,  3360  items
Sparsity:  0.0169446316262354


In [26]:
import numpy as np
sampled_uids = list(set(filtered_df['user_id']))
sampled_uids = np.random.choice(sampled_uids, 100, replace=False)
filtered_df = filtered_df[filtered_df['user_id'].isin(sampled_uids)]
sampled_mids = list(set(filtered_df['item_id']))

uid_map = create_id_map(sampled_uids)
mid_map = create_id_map(sampled_mids)
filtered_df['uid'] = filtered_df['user_id'].map(uid_map)
filtered_df['mid'] = filtered_df['item_id'].map(mid_map)
print_sample_summary(filtered_df, 'user_id', 'item_id')

5568  ratings,  100  users,  3202  items
Sparsity:  0.017389131792629606


In [29]:
from sklearn.model_selection import train_test_split
filtered_df.to_csv('../datasets/rtr/u.data')

n_runs = 5
for i in range(n_runs):
    X = filtered_df.copy()
    X = X.sample(frac=1)
    uids = X.copy().pop('uid').to_frame()
    X_train, X_test, y_train, y_test = train_test_split(X, uids,stratify=uids, test_size=0.2)
    X_train['uid'] = y_train.values.squeeze()
    X_test['uid'] = y_test.values.squeeze()
    X_train.to_csv('../datasets/rtr/u' + str(i) + '.base')
    X_test.to_csv('../datasets/rtr/u' + str(i) + '.test')

/home/t-sdivya/.conda/envs/dm/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/t-sdivya/.conda/envs/dm/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/t-sdivya/.conda/envs/dm/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a co